# Sign Language Recognition with RNN

This notebook trains an RNN model to recognize sign language from landmark data extracted using MediaPipe.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            print(gpu)
    except RuntimeError as e:
        print('error'+ e)


PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [3]:
from numba import cuda

cuda.select_device(0)
cuda.close()
cuda.select_device(0)


<weakproxy at 0x000001E139519AD0 to Device at 0x000001E139504460>

## Load Preprocessed Data

In [4]:
df = pd.read_csv('Dataset_CSVs/transformed_data.csv')

In [5]:
print(df.head())

   frame  face_0_x  face_0_y  face_1_x  face_1_y  face_2_x  face_2_y  \
0     18  0.446865  0.556099  0.435207  0.519262  0.438583  0.531356   
1     19  0.446033  0.553307  0.436424  0.516476  0.439572  0.528051   
2     20  0.444841  0.553031  0.435007  0.517342  0.438359  0.528179   
3     21  0.438405  0.555483  0.432000  0.517451  0.435937  0.529129   
4     22  0.436116  0.555547  0.432138  0.518918  0.435548  0.530537   

   face_3_x  face_3_y  face_4_x  ...  pose_28_y  pose_29_x  pose_29_y  \
0  0.423945  0.494480  0.433858  ...   2.011694   0.634389   2.089952   
1  0.424151  0.491375  0.434981  ...   1.971428   0.645629   2.029157   
2  0.422074  0.491810  0.433355  ...   1.979363   0.668940   2.030304   
3  0.419451  0.492080  0.430299  ...   2.013071   0.677119   2.062510   
4  0.419620  0.492879  0.430561  ...   2.030979   0.676004   2.079525   

   pose_30_x  pose_30_y  pose_31_x  pose_31_y  pose_32_x  pose_32_y  label  
0   0.285221   2.081855   0.559667   2.169359   0.3

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1634 entries, 0 to 1633
Columns: 1088 entries, frame to label
dtypes: float64(1086), int64(1), object(1)
memory usage: 13.6+ MB


In [7]:
df.shape

(1634, 1088)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pickle



In [9]:


# Extract features (X) and labels (y)
X = df.drop(columns=['label', 'frame']).values  # Assuming 'label' is the target column
y = df['label'].values

# Normalize features (optional but recommended)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape X for RNN input: (samples, time steps, features)
# Example assumes 1 time step (one frame per sequence), adjust as per your RNN architecture
X = X.reshape(X.shape[0], 1, X.shape[1])

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Assuming y_train and y_val are categorical (strings or integers)
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

# One-hot encode the categorical labels
onehot_encoder = OneHotEncoder(sparse_output=False)
y_train_onehot = onehot_encoder.fit_transform(y_train_encoded.reshape(-1, 1))
y_val_onehot = onehot_encoder.transform(y_val_encoded.reshape(-1, 1))


with open('Models/label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

# Save the OneHotEncoder
with open('Models/onehot_encoder.pkl', 'wb') as file:
    pickle.dump(onehot_encoder, file)

# Define the model with appropriate output units and loss function for classification
model = Sequential([
    LSTM(units=50, input_shape=(X.shape[1], X.shape[2])),
    Dense(units=y_train_onehot.shape[1], activation='softmax')  # Adjust units and activation as per your classification needs
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train_onehot, epochs=100, batch_size=32, validation_data=(X_val, y_val_onehot))

# Evaluate the model
loss, accuracy = model.evaluate(X_val, y_val_onehot)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")


Epoch 1/100
41/41 [==============================] - 8s 24ms/step - loss: 1.3455 - accuracy: 0.6121 - val_loss: 1.0303 - val_accuracy: 0.6850
Epoch 2/100
41/41 [==============================] - 0s 8ms/step - loss: 0.9276 - accuracy: 0.7774 - val_loss: 0.8108 - val_accuracy: 0.8196
Epoch 3/100
41/41 [==============================] - 0s 6ms/step - loss: 0.7315 - accuracy: 0.8531 - val_loss: 0.6530 - val_accuracy: 0.8624
Epoch 4/100
41/41 [==============================] - 0s 6ms/step - loss: 0.5899 - accuracy: 0.8814 - val_loss: 0.5204 - val_accuracy: 0.9021
Epoch 5/100
41/41 [==============================] - 0s 6ms/step - loss: 0.4815 - accuracy: 0.8975 - val_loss: 0.4290 - val_accuracy: 0.9052
Epoch 6/100
41/41 [==============================] - 0s 6ms/step - loss: 0.3988 - accuracy: 0.9166 - val_loss: 0.3623 - val_accuracy: 0.9235
Epoch 7/100
41/41 [==============================] - 0s 6ms/step - loss: 0.3432 - accuracy: 0.9357 - val_loss: 0.3154 - val_accuracy: 0.9388
Epoch 8/100


In [10]:
model.save('Models/my_model.h5')